In [1]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

# Xopt class, Astra kekgun example

This is the class method for running Xopt.

In [2]:
from xopt import Xopt

In [3]:
# Make a proper input file. 
YAML="""
xopt: {output_path: temp, verbose: true,
  algorithm: random_sampler}
  
algorithm:
  name: random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10} 

simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator

# Note that you can call another file in the top level group:
vocs: ../templates/kekgun/kekgun_vocs.yaml

"""

In [4]:
!mkdir temp/

In [5]:
# Create object
X = Xopt(config=YAML)

# Change some things to make it run faster
X.vocs['constants']['lspch'] = True
X.vocs['constants']['ipart'] = 1000
X.vocs['constants']['zstop'] = 0.2

# Show config
X

Loading config as text



            Xopt
________________________________           

Configured: True

Config as YAML:
xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {archive_path: null, merit_f: null, astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
vocs:
  name: kekgun
  description: Test case
  simulation: astra_with_generator
  templates: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
  variables:
    sig_x: [0.05,

In [6]:
# Check random inputs 
X.random_inputs()

{'sig_x': 0.3125087039190068,
 'lt': 0.03587862531760073,
 'maxe(1)': 21.048796957165067,
 'phi(1)': -28.138426182906002,
 'maxb(1)': 0.28927980245270796,
 'maxe(2)': 3.559850900836569,
 'phi(2)': -76.1261659263858,
 'maxb(2)': 0.1013113963280232,
 'maxe(3)': 8.961632963763591,
 'maxe(4)': 15.741697433367396,
 'phi(3)': 10.696016285724198,
 'phi(4)': 4.121524091726776,
 'phi(6)': 9.07009530916448,
 'ipart': 1000,
 'lspch': True,
 'zstop': 0.2}

In [7]:
%%time
# Do a random evaluate to check that everything will run
output = X.random_evaluate()
output

CPU times: user 88.2 ms, sys: 16.6 ms, total: 105 ms
Wall time: 10.3 s


{'error': False,
 'end_mean_z': 0.2,
 'end_norm_emit_x': 1.9506e-06,
 'end_core_emit_95percent_x': 1.5555e-06,
 'end_core_emit_90percent_x': 1.2696e-06,
 'end_core_emit_80percent_x': 8.696399999999999e-07,
 'end_norm_emit_y': 2.0367999999999997e-06,
 'end_core_emit_95percent_y': 1.6228e-06,
 'end_core_emit_90percent_y': 1.356e-06,
 'end_core_emit_80percent_y': 9.5781e-07,
 'end_norm_emit_z': 8.4806,
 'end_core_emit_95percent_z': 7.376699999999999e-06,
 'end_core_emit_905percent_z': 6.5559e-06,
 'end_core_emit_80percent_z': 5.151699999999999e-06,
 'end_mean_t': 7.671900000000001e-10,
 'end_mean_x': 5.9476000000000005e-06,
 'end_sigma_x': 0.0018534,
 'end_sigma_xp': 0.0087061,
 'end_cov_x__xp': 1.612902816e-05,
 'end_mean_y': 1.9978e-05,
 'end_sigma_y': 0.0019029000000000001,
 'end_sigma_yp': 0.0089346,
 'end_cov_y__yp': 1.6994609610000002e-05,
 'end_mean_kinetic_energy': 1667400.0,
 'end_sigma_z': 0.0025594000000000003,
 'end_sigma_energy': 40941.0,
 'end_cov_z__energy': -104.4414358000

In [8]:
# These are the algorithm options
X.algorithm['options']

{'verbose': 'T', 'chunk_size': 4, 'max_samples': 10}

In [9]:
# These are the options in the evaluate function
X.simulation['options']

{'archive_path': None,
 'merit_f': None,
 'astra_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in',
 'generator_input_file': '/Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in'}

# Run Sampler using processes or threads

In [10]:
from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()

In [11]:
# Show config
X.run(executor)

Starting at time 2020-04-21T15:00:06-07:00


███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler


....4 samples completed in 0.19857 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-21-1.json
....4 samples completed in 0.14265 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-21-2.json
...

In [12]:
# Check for errors
X.population['error']

[False, False, False, False, False, False, False, False, False, False, False]

# Write this configuration

In [13]:
X.save('test.yaml')

Config written as YAML to test.yaml


In [14]:
!cat test.yaml

xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {archive_path: null, merit_f: null, astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
vocs:
  name: kekgun
  description: Test case
  simulation: astra_with_generator
  templates: {astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
  variables:
    sig_x: [0.05, 1]
    lt: [0.005, 0.07]
    maxe(1): [20, 50]
    phi(1): [-30, 30]
    maxb(1): [0, 0.4]
    m

# Run with MPI

In [15]:
!mpirun -n 4 python -m mpi4py.futures -m xopt.mpi.run test.yaml

  _   
                | |  
__  _____  _ __ | |_ 
\ \/ / _ \| '_ \| __|
 >  < (_) | |_) | |_ 
/_/\_\___/| .__/ \__|
          | |        
          |_|        

_________________________________
Parallel execution with 4 workers
Loading as YAML file: test.yaml

            Xopt
________________________________           

Configured: True

Config as YAML:
xopt: {output_path: /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp, verbose: true,
  algorithm: random_sampler}
algorithm:
  name: random_sampler
  function: xopt.sampler.random_sampler
  options: {verbose: T, chunk_size: 4, max_samples: 10}
simulation:
  name: astra_with_generator
  evaluate: astra.evaluate.evaluate_astra_with_generator
  options: {archive_path: null, merit_f: null, astra_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/kekgun.in,
    generator_input_file: /Users/chrisonian/Code/GitHub/xopt/examples/templates/kekgun/dist004.in}
vocs:
  name: kekgun
  description: Test case
  simu

# Dask 

In [16]:
from dask.distributed import Client
executor = Client()
#executor = Client(processes=True)
executor

Client Scheduler: tcp://127.0.0.1:56704 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 17.18 GB


In [17]:
# Wait a few seconds for the Dask cluster to start
from time import sleep
sleep(5)

In [18]:
# Run some more
X.run(executor)

Starting at time 2020-04-21T15:01:22-07:00


███████╗ █████╗ ███╗   ███╗██████╗ ██╗     ███████╗██████╗ 
██╔════╝██╔══██╗████╗ ████║██╔══██╗██║     ██╔════╝██╔══██╗
███████╗███████║██╔████╔██║██████╔╝██║     █████╗  ██████╔╝
╚════██║██╔══██║██║╚██╔╝██║██╔═══╝ ██║     ██╔══╝  ██╔══██╗
███████║██║  ██║██║ ╚═╝ ██║██║     ███████╗███████╗██║  ██║
╚══════╝╚═╝  ╚═╝╚═╝     ╚═╝╚═╝     ╚══════╝╚══════╝╚═╝  ╚═╝
                                                           

Xopt Random Sampler


....4 samples completed in 0.18132 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-21-5.json
....4 samples completed in 0.15038 minutes
Samples written to:
 /Users/chrisonian/Code/GitHub/xopt/examples/xopt_class/temp/sampler-2020-04-21-6.json
...

In [19]:
X.population['error']

[False, False, False, False, False, False, False, False, False, False, False]

# Load into Pandas

In [20]:
import pandas as pd
from glob import glob
import json

In [21]:
jfiles = glob('temp/*json')
jfiles

['temp/sampler-2020-04-21-6.json',
 'temp/sampler-2020-04-21-1.json',
 'temp/sampler-2020-04-21-2.json',
 'temp/sampler-2020-04-21-3.json',
 'temp/sampler-2020-04-21-4.json',
 'temp/sampler-2020-04-21-5.json']

In [22]:
# Load a file
data = json.load(open(jfiles[0]))

data['inputs']
data['outputs']
data['error']

[False, False, False, False]

In [23]:
inputs = pd.DataFrame(data['inputs'])
inputs

,sig_x,lt,maxe(1),phi(1),maxb(1),maxe(2),phi(2),maxb(2),maxe(3),maxe(4),phi(3),phi(4),phi(6),ipart,lspch,zstop
0,0.458119,0.019184,28.296020,11.927447,0.036612,0.248458,156.544670,0.142194,29.596363,2.819577,14.759856,-10.940609,-7.521021,1000,True,0.2
1,0.829987,0.036703,23.685865,15.694877,0.249237,0.830556,159.225886,0.010891,27.463231,9.444380,43.674402,-27.236831,-17.885944,1000,True,0.2
2,0.545738,0.043216,41.937983,3.789982,0.019387,17.498086,-75.726106,0.256919,20.420493,9.250528,15.486363,27.027219,25.436809,1000,True,0.2
3,0.530292,0.042600,44.715837,-16.112305,0.218165,26.915230,51.843489,0.067108,1.150015,26.411331,6.842652,-37.487604,20.773493,1000,True,0.2


In [24]:
outputs = pd.DataFrame(data['outputs'])
outputs

,error,end_mean_z,end_norm_emit_x,end_core_emit_95percent_x,end_core_emit_90percent_x,end_core_emit_80percent_x,end_norm_emit_y,end_core_emit_95percent_y,end_core_emit_90percent_y,end_core_emit_80percent_y,...,end_sigma_yp,end_cov_y__yp,end_mean_kinetic_energy,end_sigma_z,end_sigma_energy,end_cov_z__energy,end_n_particle_loss,end_total_charge,end_higher_order_energy_spread,fingerprint
0,False,0.2,5.226900e-07,3.296300e-07,2.726400e-07,1.908600e-07,5.146200e-07,3.293000e-07,2.734300e-07,1.905500e-07,...,0.003877,3.628463e-06,1685300.0,0.002108,23024.0,48.304820,0,9.990000e-11,1174.338917,79c900052c1d4cf770471a9ad5456bfb
1,False,0.2,5.484500e-07,4.139900e-07,3.525700e-07,2.610600e-07,5.437500e-07,4.138100e-07,3.543500e-07,2.621000e-07,...,0.000916,6.110171e-08,1392700.0,0.003755,37465.0,139.637112,0,9.990000e-11,627.766413,268d75f5db93ba0f8b62e2db39c8b9b4
2,False,0.2,3.401700e-07,2.637800e-07,2.266700e-07,1.694900e-07,3.384800e-07,2.648200e-07,2.276000e-07,1.687200e-07,...,0.000353,3.670521e-09,2517300.0,0.003816,21073.0,68.373064,0,9.990000e-11,347.036927,fa037a926f6fe0144604381e686651f1
3,False,0.2,3.425400e-07,2.633900e-07,2.254800e-07,1.671100e-07,3.401100e-07,2.639600e-07,2.259000e-07,1.661800e-07,...,0.000361,1.734093e-08,2603300.0,0.003170,59766.0,-187.136030,0,9.990000e-11,530.412571,cb7c2df5e3ded8b1fb13cfd2ca268a1d


# Cleanup

In [25]:
!rm -r dask-worker-space

In [26]:
!rm test.yaml

In [27]:
!rm NORRAN

In [28]:
!rm -r temp